<a href="https://colab.research.google.com/github/gong-aipel/AIFFEL_quest-cr/blob/main/exploration_07.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# 우선 설치
!pip install keras-tuner --quiet

# 이후 import
import keras_tuner as kt


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.1/129.1 kB 2.4 MB/s eta 0:00:00


In [6]:
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.datasets import cifar10
import keras_tuner as kt
import numpy as np
import os

# ✅ CIFAR-10 데이터 로드 및 정규화
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

# ✅ 모델 빌더 정의
def model_builder(hp):
    model = models.Sequential()
    model.add(layers.Input(shape=(32, 32, 3)))

    # Conv Layer 1
    hp_filters1 = hp.Choice('conv1_filters', [32, 64])
    model.add(layers.Conv2D(hp_filters1, (3, 3), activation='relu'))
    model.add(layers.MaxPooling2D((2, 2)))

    # Conv Layer 2
    hp_filters2 = hp.Choice('conv2_filters', [64, 128])
    model.add(layers.Conv2D(hp_filters2, (3, 3), activation='relu'))
    model.add(layers.MaxPooling2D((2, 2)))

    # Conv Layer 3
    model.add(layers.Conv2D(64, (3, 3), activation='relu'))
    model.add(layers.Flatten())

    # Dense Layer
    hp_units = hp.Int('dense_units', min_value=32, max_value=128, step=32)
    model.add(layers.Dense(units=hp_units, activation='relu'))

    # Output Layer
    model.add(layers.Dense(10))

    # Compile
    hp_learning_rate = hp.Choice('learning_rate', [1e-2, 1e-3])
    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=hp_learning_rate),
                  loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                  metrics=['accuracy'])
    return model

# ✅ 튜너 설정
tuner = kt.Hyperband(model_builder,
                     objective='val_accuracy',
                     max_epochs=5,
                     factor=3,
                     directory='tuner_dir',
                     project_name='cifar10')

# ✅ 콜백 설정 (학습 로그 간소화)
stop_early = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=3)

# ✅ 튜닝 수행
tuner.search(x_train, y_train, epochs=10, validation_split=0.1, callbacks=[stop_early])

# ✅ 최적 모델 가져오기
best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]
print(f"Best hyperparameters: {best_hps.values}")

# ✅ 최적 모델 재학습
model = tuner.hypermodel.build(best_hps)
model.fit(x_train, y_train, epochs=10, validation_split=0.1)



Reloading Tuner from tuner_dir/cifar10/tuner0.json
Best hyperparameters: {'conv1_filters': 32, 'conv2_filters': 128, 'dense_units': 32, 'learning_rate': 0.001, 'tuner/epochs': 5, 'tuner/initial_epoch': 2, 'tuner/bracket': 1, 'tuner/round': 1, 'tuner/trial_id': '0002'}
Epoch 1/10
1407/1407 ━━━━━━━━━━━━━━━━━━━━ 96s 67ms/step - accuracy: 0.3234 - loss: 1.8184 - val_accuracy: 0.5434 - val_loss: 1.2895
Epoch 2/10
1407/1407 ━━━━━━━━━━━━━━━━━━━━ 139s 65ms/step - accuracy: 0.5657 - loss: 1.2214 - val_accuracy: 0.6212 - val_loss: 1.0698
Epoch 3/10
1407/1407 ━━━━━━━━━━━━━━━━━━━━ 143s 66ms/step - accuracy: 0.6355 - loss: 1.0408 - val_accuracy: 0.6686 - val_loss: 0.9497
Epoch 4/10
1407/1407 ━━━━━━━━━━━━━━━━━━━━ 140s 65ms/step - accuracy: 0.6780 - loss: 0.9251 - val_accuracy: 0.6696 - val_loss: 0.9405
Epoch 5/10
1407/1407 ━━━━━━━━━━━━━━━━━━━━ 140s 64ms/step - accuracy: 0.6992 - loss: 0.8556 - val_accuracy: 0.6844 - val_loss: 0.9136
Epoch 6/10
1407/1407 ━━━━━━━━━━━━━━━━━━━━ 92s 65ms/step - accuracy:

In [7]:
# ✅ SavedModel 저장
saved_model_dir = "cifar10_best_model.keras"
model.save(saved_model_dir)
model = tf.keras.models.load_model(saved_model_dir)

# ✅ TFLite 변환 및 저장
#'/content'
converter = tf.lite.TFLiteConverter.from_keras_model(model) # 모델
#converter = tf.lite.TFLiteConverter.from_saved_model(saved_model_dir)
tflite_model = converter.convert()
with open("cifar10_best_model.tflite", "wb") as f:
    f.write(tflite_model)

# ✅ TFLite 서명 확인
interpreter = tf.lite.Interpreter(model_path="cifar10_best_model.tflite")
interpreter.allocate_tensors()
print("TFLite inputs:", interpreter.get_input_details())
print("TFLite outputs:", interpreter.get_output_details())

Saved artifact at '/tmp/tmp1fctiqo8'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 32, 32, 3), dtype=tf.float32, name='input_layer_2')
Output Type:
  TensorSpec(shape=(None, 10), dtype=tf.float32, name=None)
Captures:
  133200874565328: TensorSpec(shape=(), dtype=tf.resource, name=None)
  133200874554384: TensorSpec(shape=(), dtype=tf.resource, name=None)
  133200874563216: TensorSpec(shape=(), dtype=tf.resource, name=None)
  133200874553616: TensorSpec(shape=(), dtype=tf.resource, name=None)
  133200874554576: TensorSpec(shape=(), dtype=tf.resource, name=None)
  133200874565520: TensorSpec(shape=(), dtype=tf.resource, name=None)
  133200874553808: TensorSpec(shape=(), dtype=tf.resource, name=None)
  133200874568784: TensorSpec(shape=(), dtype=tf.resource, name=None)
  133200874569168: TensorSpec(shape=(), dtype=tf.resource, name=None)
  133200874568208: TensorSpec(shape=(), dtype=tf.resource, name=None)
TFLite inputs: [